In [1]:
import numpy as np
import pandas as pd
import datetime as dt

### singular spectrum analysis（SSA）

In [35]:
def SSA(series, windowLen):
    """
    decompose the time series by SSA
    args:
        :series (pd.Series or np.array or list) -- the target time series which you want to decompose
        :windowLen (int) -- the embedding dimension
    returns:
        :(np.array) the decomposed subseries, size is (windowLen, len(series))
        
    source: https://blog.csdn.net/weixin_40169609/article/details/124076217
    """
    seriesLen = len(series)
    K = seriesLen - windowLen + 1
    X = np.zeros((windowLen, K))
    for i in range(K):
        X[:, i] = series[i:i + windowLen]
        
    U, sigma, VT = np.linalg.svd(X, full_matrices=False)

    for i in range(VT.shape[0]):
        VT[i, :] *= sigma[i]
    A = VT
    
    subSeries = np.zeros((windowLen, seriesLen))
    for i in range(windowLen):
        for j in range(windowLen-1):
            for m in range(j+1):
                subSeries[i, j] += A[i, j-m] * U[m, i]
            subSeries[i, j] /= (j+1)
        for j in range(windowLen-1, seriesLen - windowLen + 1):
            for m in range(windowLen):
                subSeries[i, j] += A[i, j-m] * U[m, i]
            subSeries[i, j] /= windowLen
        for j in range(seriesLen - windowLen + 1, seriesLen):
            for m in range(j-seriesLen+windowLen, windowLen):
                subSeries[i, j] += A[i, j - m] * U[m, i]
            subSeries[i, j] /= (seriesLen - j)
    return subSeries

In [43]:
"""
Usages:
""" 
series = pd.Series(np.random.randn(20),index=pd.date_range(dt.datetime(2022,1,1),periods=20))
windowLen = 10
subSeries = SSA(series, windowLen)

### EMD and its variants

In [46]:
"""
PyEMD includes EMD, EEMD, CEEMDAN.
Usages:
"""
import PyEMD

series = pd.Series(np.random.randn(20),index=pd.date_range(dt.datetime(2022,1,1),periods=20))
emd = EMD()
subSeries = emd(series.values)

### wavelet transform (WT)

In [41]:
import pywt

def wavelet_packet_three_level(series):
    """
    decompose the time series by WT
    args:
        :series (pd.Series or np.array or list) -- the target time series which you want to decompose
    returns:
        :(np.array) the decomposed subseries
    
    source: https://zhuanlan.zhihu.com/p/441921299
    """
    
    mother_wavelet = 'dmey'
    wp = pywt.WaveletPacket(data=series, wavelet=mother_wavelet, mode='symmetric', maxlevel=3)
    node_name_list = [node.path for node in wp.get_level(3, 'natural')]
    rec_results = []
    for i in node_name_list:
        new_wp = pywt.WaveletPacket(data=np.zeros(len(series)), wavelet=mother_wavelet, mode='symmetric',maxlevel=3)
        new_wp[i] = wp[i].data
        x_i = new_wp.reconstruct(update=True)
        rec_results.append(x_i)

    subSeries = np.array(rec_results)

    return subSeries

In [52]:
"""
Usages:
""" 
series = pd.Series(np.random.randn(20),index=pd.date_range(dt.datetime(2022,1,1),periods=20))
subSeries = wavelet_packet_three_level(series)

### variational mode decomposition (VMD)

In [2]:
"""
source: https://blog.csdn.net/abc1234abcdefg/article/details/123319043

Usages:
"""
from vmdpy import VMD

alpha = 2000      # moderate bandwidth constraint  
tau = 0.            # noise-tolerance (no strict fidelity enforcement)  
K = 5        # 5 modes  
DC = 0            # no DC part imposed  
init = 1           # initialize omegas uniformly  
tol = 1e-7

series = pd.Series(np.random.randn(20),index=pd.date_range(dt.datetime(2022,1,1),periods=20))
subSeries,_,_ = VMD(series, alpha, tau, K, DC, init, tol)